In [1]:
import pandas as pd
from IPython.display import clear_output
import ipywidgets as widgets
from IPython.display import display
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [2]:
data = pd.read_csv('data.csv')
data

,nombre,apellido,edad,profesion,sueldo
0,Ana,García,28,Ingeniera de Software,45000
1,Carlos,Rodríguez,35,Médico,75000
2,Laura,Martínez,42,Abogada,62000
3,Javier,López,29,Diseñador Gráfico,38000
4,María,Fernández,51,Profesora,48000
5,Pedro,Sánchez,33,Contador,52000
6,Sofía,Ramírez,26,Periodista,35000
7,Diego,González,45,Arquitecto,68000
8,Elena,Torres,39,Psicóloga,55000
9,Miguel,Vargas,31,Analista de Datos,58000


In [3]:
nombre_ana =  data[data["nombre"] == "Ana"]
nombre_ana

,nombre,apellido,edad,profesion,sueldo
0,Ana,García,28,Ingeniera de Software,45000


In [4]:
columnas = data.columns.to_list()
columnas

['nombre', 'apellido', 'edad', 'profesion', 'sueldo']

In [5]:
def parametro_busqueda_keyword(parametro):
    res =  data[parametro]
    print(res)


In [6]:


def user_input(debug):

    if debug:
        return "dime unas profesiones"
    else:
        text_input = widgets.Text(
            value='',
            description='Tu pregunta:',
            placeholder='Escribe tu pregunta aquí'
        )
        
        display(text_input)
        
        def handle_submit(sender):
            global user_question
            user_question = sender.value
            clear_output()
            print(f"Pregunta ingresada: {user_question}")
        
        text_input.observe(handle_submit, names='value')
        
        return text_input.value

In [7]:

checkpoint = "bigscience/bloomz-1b7"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True
)


`torch_dtype` is deprecated! Use `dtype` instead!


In [8]:
prompt = f"""INSTRUCCIÓN: Elige UNA palabra de esta lista [{', '.join(columnas)}] que mejor clasifique la siguiente frase.
FRASE: '{user_input(True)}'
IMPORTANTE: Solo responde con una palabra de la lista. Nada más.
RESPUESTA:"""

inputs = tokenizer.encode(prompt, return_tensors="pt")
attention_mask = torch.ones_like(inputs)

outputs = model.generate(
    inputs,
    attention_mask=attention_mask,
    max_new_tokens=2,
    num_return_sequences=1,
    do_sample=True,
    temperature=0.7,
    top_k=5,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.2
)

res = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
words = res.split()
for word in reversed(words):
    word = word.strip("'\".,!? ")
    if word in columnas:
        last_word = word
        break
    else:
        last_word = "nombre"

print(f"Palabra clasificada: {last_word}")
parametro_busqueda_keyword(str(last_word))

Palabra clasificada: profesion
0     Ingeniera de Software
1                    Médico
2                   Abogada
3         Diseñador Gráfico
4                 Profesora
5                  Contador
6                Periodista
7                Arquitecto
8                 Psicóloga
9         Analista de Datos
10                Enfermera
11          Ingeniero Civil
12        Marketing Digital
13    Director de Proyectos
14       Desarrolladora Web
15     Consultor Financiero
16         Recursos Humanos
17       Ingeniero Mecánico
18        Community Manager
19        Gerente de Ventas
Name: profesion, dtype: object
